<a href="https://colab.research.google.com/github/joshiayush/llm/blob/main/notebooks/llm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Dependencies Installation

In [2]:
!pip install requests
!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 23.7 MB/s eta 0:00:00


# Tokenization

In [3]:
import requests

In [4]:
raw_text = requests.get("https://raw.githubusercontent.com/rasbt/LLMs-from-scratch/refs/heads/main/ch02/01_main-chapter-code/the-verdict.txt")
raw_text = raw_text.content.decode()

## BYTE PAIR ENCODING

**Advantages:**
* Reduces vocab lenght.
* Handles out-of-vocabulary words.

### Note

__[BOS] (beginning of sequence):__ This token marks the start of a text. It
signifies to the LLM where a piece of content begins.

__[EOS] (end of sequence):__ This token is positioned at the end of a text,
and is especially useful when concatenating multiple unrelated texts,
similar to `<|endoftext|>`. For instance, when combining two different
Wikipedia articles or books, the __[EOS]__ token indicates where one article
ends and the next one begins.

__[PAD] (padding):__ When training LLMs with batch sizes larger than one,
the batch might contain texts of varying lengths. To ensure all texts have
the same length, the shorter texts are extended or "padded" using the
__[PAD]__ token, up to the length of the longest text in the batch.

Note that the tokenizer used for GPT models does not need any of these tokens mentioned above but only uses an `<|endoftext|>` token for simplicity.

The tokenizer used for GPT models also doesn't use an `<|unk|>` token for __out-of-vocabulary__ words. Instead, GPT models use a byte pair encoding tokenizer, which breaks down words into subword units.


In [5]:
import tiktoken

Our BPE tokenizer.

In [6]:
tokenizer = tiktoken.get_encoding("gpt2")

In [7]:
enc_text = tokenizer.encode(raw_text)

### Implementing a Data Loader

In [10]:
from tiktoken.core import Encoding
from torch.utils.data import Dataset, DataLoader

In [11]:
class GPTDatasetV1(Dataset):

    def __init__(self, txt: str, tokenizer: Encoding, max_length: int, stride: int):
        self.x = list()
        self.y = list()

        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})

        for i in range(0, len(token_ids) - max_length, stride):
            x_chunk = token_ids[i : i + max_length]
            y_chunk = token_ids[i + 1 : i + max_length + 1]
            self.x.append(torch.tensor(x_chunk))
            self.y.append(torch.tensor(y_chunk))

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

In [12]:
def create_dataloader_v1(
    txt: str,
    batch_size: int = 4,
    max_length: int = 256,
    stride: int = 128,
    shuffle: bool = True,
    drop_last: bool = True,
    num_workers: int = 0,
):
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)

    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers,
    )
    return dataloader

In [15]:
dataloader = create_dataloader_v1(
    raw_text, batch_size=1, max_length=4, stride=1, shuffle=False
)

In [19]:
iter = iter(dataloader)
batch_1 = next(iter)

We can look at our first batch.

In [20]:
print(batch_1)

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]


And here is the second batch.

In [21]:
batch_2 = next(iter)

In [22]:
print(batch_2)

[tensor([[ 367, 2885, 1464, 1807]]), tensor([[2885, 1464, 1807, 3619]])]


**CREATE TOKEN EMBEDDINGS**

In [23]:
import torch

In [ ]:
input_ids = torch.tensor([2, 3, 5, 1])


In [ ]:
vocab_size = 6
output_dim = 3

torch.manual_seed(123)
embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

In [ ]:
print(embedding_layer.weight)


Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)


In [ ]:
print(embedding_layer(torch.tensor([3])))


tensor([[-0.4015,  0.9666, -1.1481]], grad_fn=<EmbeddingBackward0>)


In [ ]:
print(embedding_layer(input_ids))


tensor([[ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-2.8400, -0.7849, -1.4096],
        [ 0.9178,  1.5810,  1.3010]], grad_fn=<EmbeddingBackward0>)


**POSITIONAL EMBEDDINGS (ENCODING WORD POSITIONS)**

In [ ]:
vocab_size = 50257
output_dim = 256

token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

In [ ]:
max_length = 4
dataloader = create_dataloader_v1(
    raw_text, batch_size=8, max_length=max_length,
    stride=max_length, shuffle=False
)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)

In [ ]:
print("Token IDs:\n", inputs)
print("\nInputs shape:\n", inputs.shape)

Token IDs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Inputs shape:
 torch.Size([8, 4])


In [ ]:
token_embeddings = token_embedding_layer(inputs)
print(token_embeddings.shape)

torch.Size([8, 4, 256])


In [ ]:
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)

In [ ]:
pos_embeddings = pos_embedding_layer(torch.arange(max_length))
print(pos_embeddings.shape)

torch.Size([4, 256])


In [ ]:
input_embeddings = token_embeddings + pos_embeddings
print(input_embeddings.shape)

torch.Size([8, 4, 256])
